In [ ]:
from adjustText import adjust_text
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pystan
import seaborn as sns

import premierleague as p

%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 5)
plt.rcParams['font.size'] = 13

# Exploration of simple extension of D&C

In [ ]:
data, prior_data, teams = p.load_data()
data.head()

In [ ]:
sns.countplot(data['FTHG']);

In [ ]:
sns.countplot(data['FTAG']);

In [ ]:
counts = data.groupby(['FTHG', 'FTAG']).count()['Div'].unstack().fillna(0)

fig, ax = plt.subplots(figsize=(10, 10))
img = ax.imshow(counts, origin='lower', cmap="Blues")
ax.set_xticks(data['FTAG'].unique())
ax.set_yticks(data['FTHG'].unique())
ax.set_xlabel("Away Goals")
ax.set_ylabel("Home Goals")
plt.colorbar(img, orientation='horizontal', ax=ax, label="Number of matches");

In [ ]:
avg_scored = .5*(data.groupby('HomeTeam')['FTHG'].mean() + data.groupby('AwayTeam')['FTAG'].mean())
avg_conceded = .5*(data.groupby('HomeTeam')['FTAG'].mean() + data.groupby('AwayTeam')['FTHG'].mean())

fig, ax = plt.subplots()
ax.scatter(avg_scored, avg_conceded)
texts = []
for ai,bi,s in zip(avg_scored, avg_conceded, teams):
    texts.append(plt.text(ai, bi, s, size=9))
adjust_text(texts, arrowprops=dict(arrowstyle="-", color='k', lw=0.))
plt.xlabel("Average goals scored")
plt.ylabel("Average goals conceded");

In [ ]:
fit = p.fit_model("premierleague_1.stan", chains=4, iter=5000, seed=42)

In [ ]:
home_goals = fit['home_goals_sim']
away_goals = fit['away_goals_sim']

def plot_result_counts(ax, i=None, cmap="Reds"):
    # plot the distribution of scorelines
    if i is None:
        counts = data.groupby(['FTHG', 'FTAG']).count()['Div'].unstack().fillna(0)
    else:
        df = pd.DataFrame({'FTHG': home_goals[i], 'FTAG': away_goals[i], 'dummy': np.arange(380)})
        counts = df.groupby(['FTHG', 'FTAG']).count()['dummy'].unstack().fillna(0)
    pads = [10 - si for si in counts.shape]
    counts = np.pad(counts, pad_width=((0, pads[0]), (0, pads[1])), mode='constant', constant_values=0) 
    ax.imshow(counts, origin='lower', cmap=cmap)
    ax.set_xticks([])
    ax.set_yticks([])

fig, ax = plt.subplots(4, 4, figsize=(10, 10))
i = np.random.randint(0, home_goals.shape[0], 15)
for j, axi in enumerate(ax.ravel()):
    k = i[j-1] if j!=0 else None
    cmap = "Reds" if j!=0 else "Blues"
    plot_result_counts(axi, i=k, cmap=cmap)
fig.text(0.47, 0.07, "Away goals")
fig.text(0.07, 0.55, "Home goals", rotation=90);

In [ ]:
fig, ax = plt.subplots(4, 4, figsize=(10, 10))
for j, axi in enumerate(ax.ravel()):
    k = i[j-1] if j!=0 else None
    if j==0:
        sns.countplot(data['FTHG'].values, color='b', ax=axi)
    else:
        sns.countplot(home_goals[k], color='r', ax=axi)
    axi.set_xlim([-1, 10])
    axi.set_xticks([])
    axi.set_yticks([])
    axi.set_ylabel("")
fig.text(0.45, 0.07, "Home goals");

In [ ]:
fig, ax = plt.subplots(4, 4, figsize=(10, 10))
for j, axi in enumerate(ax.ravel()):
    k = i[j-1] if j!=0 else None
    if j==0:
        sns.countplot(data['FTAG'].values, color='b', ax=axi)
    else:
        sns.countplot(away_goals[k], color='r', ax=axi)
    axi.set_xlim([-1, 10])
    axi.set_xticks([])
    axi.set_yticks([])
    axi.set_ylabel("")
fig.text(0.45, 0.07, "Away goals");

Dixon & Coles suggest that there's some correlation between the home and away scorelines for low scoring matches. They test this by computing the empirical probabilities (i.e. counts/total) for each team scoring 0 or 1 goals, and divide by the product of the marginals. Following their approach (but Bayesian), I define the following test statistic:

$$T(y) = \frac{N_\mathrm{games}}{4}\sum\limits_{i=0}^1\sum\limits_{j=0}^1 \dfrac{N_\mathrm{joint}(i, j)}{N_\mathrm{home}(i)N_\mathrm{away}(j)},$$

where $N_\mathrm{joint}(i, j)$ is the number of games where the home team scored $i$ goals and the away team scored $j$ goals, $N_\mathrm{home}(i)$ is the number of games where the home team scored $i$ goals and $N_\mathrm{away}(j)$ is the number of games where the away team scored $j$ goals.

In [ ]:
counts = data.groupby(['FTHG', 'FTAG']).count()['Div'].unstack().fillna(0).as_matrix()
T_corr = np.array(
    [380*counts[i, j]/(counts[i, :].sum()*counts[:, j].sum()) for i in [0,1] for j in [0,1]]
).sum()/4

def T_corr_rep_fn(i):
    df = pd.DataFrame({'FTHG': home_goals[i], 'FTAG': away_goals[i], 'dummy': np.arange(380)})
    counts = df.groupby(['FTHG', 'FTAG']).count()['dummy'].unstack().fillna(0).as_matrix()
    return np.array(
        [380*counts[i, j]/(counts[i, :].sum()*counts[:, j].sum()) for i in [0,1] for j in [0,1]]
    ).sum()/4

T_corr_rep = np.array([T_corr_rep_fn(i) for i in range(home_goals.shape[0])])
p_corr = (T_corr_rep >= T_corr).sum() / len(T_corr_rep)

sns.distplot(T_corr_rep, kde=False)
plt.axvline(T_corr, c='k', ls='--')
plt.xlabel(r"$T(y)$")
plt.text(0.8, 550, r"$p = {:.2f}$".format(p_corr), fontsize=15)

The observed outcome is not in huge tension with the data, although the $p$-value is somewhat close to 1. Given the fact that someone else pointed this out for a *different* set of football data, perhaps we should lend more weight to this result. I also suspect that the highest number of goals scored is a bit extreme in the model as well.

In [ ]:
T_max = data[['FTAG', 'FTHG']].max().max()
T_max_rep = np.max(
    np.concatenate((home_goals, away_goals), axis=-1), axis=1
)

sns.countplot(T_max_rep)
plt.axvline(2, c='k', ls='--')
plt.xlabel(r"$T(y)$");

Nope! It's fine. The test statistic looks Poisson distributed, which is probably what we'd expect (?) Deep breath...time to estimate generalisation errors. 

In [ ]:
data = data.set_index('Date')
data.index = pd.to_datetime(data.index)
data['Gameweek'] = data.index.year * 100 + data.index.week
data['Gameweek'] = data['Gameweek'].diff(1).fillna(1.)
data.loc[data['Gameweek'] != 0, 'Gameweek'] = 1.
data['Gameweek'] = data['Gameweek'].cumsum().astype(int)

In [ ]:
data_uptofinal = data[data['Gameweek'] < data['Gameweek'].max()]
data_uptofinal_stan = {'nteams': len(teams),
                       'ngames': len(data_uptofinal),
                       'home_goals': data_uptofinal['FTHG'].values,
                       'away_goals': data_uptofinal['FTAG'].values,
                       'home_team': data_uptofinal['HomeTeam'].values,
                       'away_team': data_uptofinal['AwayTeam'].values}
sm = pystan.StanModel(file="premierleague_1.stan")
fit_uptofinal = sm.sampling(data=data_uptofinal_stan, chains=4, iter=3000, seed=42)

In [ ]:
home_goals = fit['home_goals_sim']
away_goals = fit['away_goals_sim']

prob_home_win = (home_goals > away_goals).sum(axis=0) / home_goals.shape[0]
prob_away_win = (home_goals < away_goals).sum(axis=0) / home_goals.shape[0]
prob_draw = 1. - prob_home_win - prob_away_win

data['model_home'] = prob_home_win
data['model_away'] = prob_away_win
data['model_draw'] = prob_draw